In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_npz = np.load('/content/drive/MyDrive/Colab Notebooks/train.npz')
train_data = train_npz['data']
test_npz  = np.load('/content/drive/MyDrive/Colab Notebooks/test_input.npz')
test_data  = test_npz['data']

In [3]:
print(train_data.shape, test_data.shape)

# Split once for later use
X_train = train_data[..., :50, :]
Y_train = train_data[:, 0, 50:, :2]

(10000, 50, 110, 6) (2100, 50, 50, 6)


In [4]:
class TrajectoryDatasetTrain(Dataset):
    def __init__(self, data, scale=10.0, augment=True):
        """
        data: Shape (N, 50, 110, 6) Training data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        augment: Whether to apply data augmentation (only for training)
        """
        self.data = data
        self.scale = scale
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx]
        # Getting 50 historical timestamps and 60 future timestamps
        hist = scene[:, :50, :].copy()    # (agents=50, time_seq=50, 6)
        future = torch.tensor(scene[0, 50:, :2].copy(), dtype=torch.float32)  # (60, 2)

        # Data augmentation(only for training)
        if self.augment:
            if np.random.rand() < 0.5:
                theta = np.random.uniform(-np.pi, np.pi)
                R = np.array([[np.cos(theta), -np.sin(theta)],
                              [np.sin(theta),  np.cos(theta)]], dtype=np.float32)
                # Rotate the historical trajectory and future trajectory
                hist[..., :2] = hist[..., :2] @ R
                hist[..., 2:4] = hist[..., 2:4] @ R
                future = future @ R
            if np.random.rand() < 0.5:
                hist[..., 0] *= -1
                hist[..., 2] *= -1
                future[:, 0] *= -1

        # Use the last timeframe of the historical trajectory as the origin
        origin = hist[0, 49, :2].copy()  # (2,)
        hist[..., :2] = hist[..., :2] - origin
        future = future - origin

        # Normalize the historical trajectory and future trajectory
        hist[..., :4] = hist[..., :4] / self.scale
        future = future / self.scale

        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            y=future.type(torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )

        return data_item


class TrajectoryDatasetTest(Dataset):
    def __init__(self, data, scale=10.0):
        """
        data: Shape (N, 50, 110, 6) Testing data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        """
        self.data = data
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Testing data only contains historical trajectory
        scene = self.data[idx]  # (50, 50, 6)
        hist = scene.copy()

        origin = hist[0, 49, :2].copy()
        hist[..., :2] = hist[..., :2] - origin
        hist[..., :4] = hist[..., :4] / self.scale

        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )
        return data_item

In [36]:
# angel correct
# class TrajectoryDatasetTrain(Dataset):
#     def __init__(self, data, scale=10.0, augment=True):
#         self.data = data
#         self.scale = scale
#         self.augment = augment

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         scene = self.data[idx]
#         hist = scene[:, :50, :].copy()    # (agents=50, time_seq=50, 6)
#         future = torch.tensor(scene[0, 50:, :2].copy(), dtype=torch.float32)  # (60, 2)

#         # Data augmentation(only for training)
#         if self.augment:
#             if np.random.rand() < 0.5:
#                 theta = np.random.uniform(-np.pi, np.pi)
#                 R = np.array([[np.cos(theta), -np.sin(theta)],
#                               [np.sin(theta),  np.cos(theta)]], dtype=np.float32)
#                 # Rotate the historical trajectory and future trajectory
#                 hist[..., :2] = hist[..., :2] @ R
#                 hist[..., 2:4] = hist[..., 2:4] @ R
#                 future = future @ R
#             if np.random.rand() < 0.5:
#                 hist[..., 0] *= -1
#                 hist[..., 2] *= -1
#                 future[:, 0] *= -1

#         # Use the last timeframe of the historical trajectory as the origin
#         origin = hist[0, 49, :2].copy()  # (2,)
#         hist[..., :2] = hist[..., :2] - origin
#         future = future - origin

#         # =========== 关键：将所有agent对齐ego agent的朝向 =============
#         # 使用ego最后一帧的速度方向作为朝向参考
#         ego_v = hist[0, 49, 2:4].copy()  # (vx, vy)
#         norm = np.linalg.norm(ego_v)
#         if norm > 1e-3:
#             theta = np.arctan2(ego_v[1], ego_v[0])   # ego当前朝向角（相对x轴）
#             R = np.array([
#                 [np.cos(-theta), -np.sin(-theta)],
#                 [np.sin(-theta),  np.cos(-theta)]
#             ], dtype=np.float32)
#             hist[..., :2] = hist[..., :2] @ R
#             hist[..., 2:4] = hist[..., 2:4] @ R
#             future = future @ R
#         # ============================================================

#         # Normalize the historical trajectory and future trajectory
#         hist[..., :4] = hist[..., :4] / self.scale
#         future = future / self.scale

#         data_item = Data(
#             x=torch.tensor(hist, dtype=torch.float32),
#             y=future.type(torch.float32),
#             origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
#             scale=torch.tensor(self.scale, dtype=torch.float32),
#         )

#         return data_item


# class TrajectoryDatasetTest(Dataset):
#     def __init__(self, data, scale=10.0):
#         """
#         data: shape (N, 50, 110, 6)
#         scale: normalization factor (default: 10.0)
#         """
#         self.data = data
#         self.scale = scale

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         scene = self.data[idx]  # (50, 110, 6)
#         hist = scene[:, :50, :].copy()  # 只取前 50 帧历史，shape (50, 50, 6)

#         # 平移使 ego agent 的最后一帧作为原点
#         origin = hist[0, 49, :2].copy()
#         hist[..., :2] -= origin

#         # 对齐朝向：让 ego agent 的速度方向对齐 x 轴
#         theta = 0.0
#         ego_v = hist[0, 49, 2:4].copy()
#         norm = np.linalg.norm(ego_v)
#         if norm > 1e-3:
#             theta = np.arctan2(ego_v[1], ego_v[0])
#             R = np.array([
#                 [np.cos(-theta), -np.sin(-theta)],
#                 [np.sin(-theta),  np.cos(-theta)]
#             ], dtype=np.float32)
#             hist[..., :2] = hist[..., :2] @ R
#             hist[..., 2:4] = hist[..., 2:4] @ R

#         # 缩放归一化
#         hist[..., :4] /= self.scale

#         # 构造 PyG 图结构（单个样本 = 一整张图）
#         data_item = Data(
#             x=torch.tensor(hist, dtype=torch.float32),  # (50, 50, 6)
#             origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),  # (1, 2)
#             scale=torch.tensor(self.scale, dtype=torch.float32),
#             theta=torch.tensor(theta, dtype=torch.float32).unsqueeze(0)  # (1,)
#         )
#         return data_item

In [5]:
torch.manual_seed(251)
np.random.seed(42)

scale = 1.0

N = len(train_data)
val_size = int(0.1 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using CUDA GPU


In [6]:
import torch

In [7]:
class LSTM(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=128, output_dim=60 * 2):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x = data.x
        x= x.reshape(-1, 50, 50, 6)  # (batch_size, num_agents, seq_len, input_dim)
        x = x[:, 0, :, :] # Only Consider ego agent index 0

        lstm_out, _ = self.lstm(x)
        # lstm_out is of shape (batch_size, seq_len, hidden_dim) and we want the last time step output
        out = self.fc(lstm_out[:, -1, :])
        return out.view(-1, 60, 2)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv

class ImprovedLSTM_GCN(nn.Module):
    """
    升级版 LSTM+GCN 轨迹预测器
    """
    def __init__(self, input_dim=6, hidden_dim=128, pred_len=60, gcn_out_dim=32, dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.pred_len = pred_len

        # LSTM用于ego历史序列
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=dropout
        )

        # GCN用于全局最后一帧邻居关系
        self.gcn1 = GCNConv(input_dim, 32)
        self.gcn2 = GCNConv(32, gcn_out_dim)

        # 全连接
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim + gcn_out_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, pred_len * 2)
        )

    def build_graph_batch(self, x_last, max_neighbor_dist=None):
        """
        x_last: (B, N, 6)
        Returns a PyG Batch object.
        """
        B, N, C = x_last.shape
        data_list = []
        for b in range(B):
            node_feat = x_last[b]  # (N,6)
            src, dst = [], []
            for i in range(N):
                for j in range(N):
                    if i == j:
                        continue
                    # 可选：只连距离小于阈值的邻居
                    if max_neighbor_dist is not None:
                        pos_i = node_feat[i, :2]
                        pos_j = node_feat[j, :2]
                        dist = torch.norm(pos_i - pos_j)
                        if dist > max_neighbor_dist:
                            continue
                    src.append(i)
                    dst.append(j)
            edge_index = torch.tensor([src, dst], dtype=torch.long, device=node_feat.device)
            data = Data(x=node_feat, edge_index=edge_index)
            data_list.append(data)
        batch = Batch.from_data_list(data_list)
        return batch

    def forward(self, data):
        """
        data.x: (B*A, T, D) or (B, A, T, D)
        data.num_graphs: B
        """
        x = data.x
        B = data.num_graphs

        # —— 1) 恢复成 (B, A=50, T, D) ——
        x = x.view(B, 50, 50, -1)  # (B,50,50,D)

        # —— 2) 提取 ego agent 的历史序列 ——
        ego_hist = x[:, 0, :, :]  # (B, 50, D)

        # —— 3) LSTM 编码（两层）——
        lstm_out, _ = self.lstm(ego_hist)       # (B,50,hidden_dim)
        last_hidden = lstm_out[:, -1, :]        # (B, hidden_dim)

        # —— 4) 用GCN建图对最后一帧所有agent聚合 ——
        x_last = x[:, :, -1, :6]  # (B,50,6)
        graph_batch = self.build_graph_batch(x_last)
        h = F.relu(self.gcn1(graph_batch.x, graph_batch.edge_index))
        h = self.gcn2(h, graph_batch.edge_index)
        h_nodes = h.view(B, 50, -1)  # (B, 50, gcn_out_dim)
        ego_gcn = h_nodes[:, 0, :]   # (B, gcn_out_dim)



        # —— 5) 拼接 LSTM 和 GCN ——

        fusion = torch.cat([last_hidden, ego_gcn], dim=-1)  # (B, hidden_dim+gcn_out_dim)


        # —— 6) 全连接预测 ——
        out = self.fc(fusion)                      # (B, pred_len*2)
        out = out.view(B, self.pred_len, 2)        # (B, pred_len, 2)
        return out
# ====== 测试用例 ======
if __name__ == "__main__":
    class DummyData:
        pass

    B, A, T, D = 4, 50, 50, 6
    pred_len = 60
    torch.manual_seed(123)
    dummy = DummyData()
    dummy.x = torch.randn(B*A, T, D)
    dummy.num_graphs = B

    model = ImprovedLSTM_GCN(input_dim=6, hidden_dim=128, pred_len=pred_len, gcn_out_dim=32, dropout=0.1)
    output = model(dummy)
    print(output.shape)  # (B, pred_len, 2)

torch.Size([4, 60, 2])


In [12]:
#without angel correct
model = ImprovedLSTM_GCN(
    input_dim=6,
    hidden_dim=256,
    pred_len=60,
    dropout=0.1,
).to(device)


optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.75) # You can try different schedulers
early_stopping_patience = 10
best_val_loss = float('inf')
no_improvement = 0
criterion = nn.MSELoss()

In [13]:
for epoch in tqdm.tqdm(range(100), desc="Epoch", unit="epoch"):
    # ---- Training ----
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(batch.num_graphs, 60, 2)
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        train_loss += loss.item()

    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = batch.to(device)
            pred = model(batch)
            y = batch.y.view(batch.num_graphs, 60, 2)
            val_loss += criterion(pred, y).item()

            # show MAE and MSE with unnormalized data
            pred = pred * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            y = y * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            val_mae += nn.L1Loss()(pred, y).item()
            val_mse += nn.MSELoss()(pred, y).item()

    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    scheduler.step()
    # scheduler.step(val_loss)

    tqdm.tqdm.write(f"Epoch {epoch:03d} | Learning rate {optimizer.param_groups[0]['lr']:.6f} | train normalized MSE {train_loss:8.4f} | val normalized MSE {val_loss:8.4f}, | val MAE {val_mae:8.4f} | val MSE {val_mse:8.4f}")
    if val_loss < best_val_loss - 1e-3:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        no_improvement += 1
        if no_improvement >= early_stopping_patience:
            print("Early stop!")
            break

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]<ipython-input-7-2409732783>:30: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future @ R
<ipython-input-7-2409732783>:39: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future - origin
Epoch:   1%|          | 1/100 [00:16<26:50, 16.27s/epoch]

Epoch 000 | Learning rate 0.001000 | train normalized MSE 170.9074 | val normalized MSE  60.5126, | val MAE   4.8404 | val MSE  60.5126


Epoch:   2%|▏         | 2/100 [00:31<25:15, 15.47s/epoch]

Epoch 001 | Learning rate 0.001000 | train normalized MSE  34.8927 | val normalized MSE  29.9572, | val MAE   3.2393 | val MSE  29.9572


Epoch:   3%|▎         | 3/100 [00:46<24:35, 15.21s/epoch]

...

Epoch 061 | Learning rate 0.000422 | train normalized MSE   8.9580 | val normalized MSE   8.9165, | val MAE   1.4721 | val MSE   8.9165


Epoch:  63%|██████▎   | 63/100 [15:51<09:32, 15.46s/epoch]

Epoch 062 | Learning rate 0.000422 | train normalized MSE   9.0880 | val normalized MSE   8.5480, | val MAE   1.3855 | val MSE   8.5480


Epoch:  63%|██████▎   | 63/100 [16:07<09:28, 15.35s/epoch]

Epoch 063 | Learning rate 0.000422 | train normalized MSE   9.1467 | val normalized MSE   8.7322, | val MAE   1.4184 | val MSE   8.7322
Early stop!


In [14]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

best_model = torch.load("best_model.pt")
model = ImprovedLSTM_GCN(
    input_dim=6,
    hidden_dim=256,
    pred_len=60,
    dropout=0.1,
).to(device)

model.load_state_dict(best_model)
model.eval()

pred_list = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        pred_norm = model(batch)

        # Reshape the prediction to (N, 60, 2)
        pred = pred_norm * batch.scale.view(-1,1,1) + batch.origin.unsqueeze(1)
        pred_list.append(pred.cpu().numpy())
pred_list = np.concatenate(pred_list, axis=0)  # (N,60,2)
pred_output = pred_list.reshape(-1, 2)  # (N*60, 2)
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])
output_df.index.name = 'index'
output_df.to_csv('submission.csv', index=True)

In [24]:
#with angel correct
model = ImprovedLSTM_GCN(
    input_dim=6,
    hidden_dim=256,
    pred_len=60,
    dropout=0.1,
).to(device)


optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.75) # You can try different schedulers
early_stopping_patience = 10
best_val_loss = float('inf')
no_improvement = 0
criterion = nn.MSELoss()

In [25]:
for epoch in tqdm.tqdm(range(100), desc="Epoch", unit="epoch"):
    # ---- Training ----
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(batch.num_graphs, 60, 2)
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        train_loss += loss.item()

    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = batch.to(device)
            pred = model(batch)
            y = batch.y.view(batch.num_graphs, 60, 2)
            val_loss += criterion(pred, y).item()

            # show MAE and MSE with unnormalized data
            pred = pred * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            y = y * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            val_mae += nn.L1Loss()(pred, y).item()
            val_mse += nn.MSELoss()(pred, y).item()

    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    scheduler.step()
    # scheduler.step(val_loss)

    tqdm.tqdm.write(f"Epoch {epoch:03d} | Learning rate {optimizer.param_groups[0]['lr']:.6f} | train normalized MSE {train_loss:8.4f} | val normalized MSE {val_loss:8.4f}, | val MAE {val_mae:8.4f} | val MSE {val_mse:8.4f}")
    if val_loss < best_val_loss - 1e-3:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        no_improvement += 1
        if no_improvement >= early_stopping_patience:
            print("Early stop!")
            break

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]<ipython-input-16-3024836103>:24: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future @ R
<ipython-input-16-3024836103>:33: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future - origin
<ipython-input-16-3024836103>:47: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future @ R
Epoch:   1%|          | 1/100 [00:17<28:40, 17.37s/epoch]

Epoch 000 | Learning rate 0.001000 | train normalized MSE 182.8710 | val normalized MSE  38.1784, | val MAE   3.7449 | val MSE  38.1784


Epoch:   2%|▏         | 2/100 [00:34<28:14, 17.29s/epoch]

Epoch 001 | Learning rate 0.001000 | train normalized MSE  34.9320 | val normalized MSE  24.8856, | val MAE   2.9198 | val MSE  24.8856


Epoch:   3%|▎         | 3/100 [00:51<27:46, 17.18s/epoch]

...

Epoch 061 | Learning rate 0.000422 | train normalized MSE   9.1337 | val normalized MSE   8.7871, | val MAE   1.4576 | val MSE   8.7871


Epoch:  63%|██████▎   | 63/100 [17:57<10:29, 17.01s/epoch]

Epoch 062 | Learning rate 0.000422 | train normalized MSE   9.1242 | val normalized MSE   8.7058, | val MAE   1.4191 | val MSE   8.7058


Epoch:  63%|██████▎   | 63/100 [18:14<10:42, 17.38s/epoch]

Epoch 063 | Learning rate 0.000422 | train normalized MSE   9.0986 | val normalized MSE   8.6675, | val MAE   1.4539 | val MSE   8.6675
Early stop!


In [45]:

class SocialLSTM(nn.Module):
    """
    简化版 Social-LSTM：所有 agent 使用共享 LSTM，
    每个时间步聚合邻居的 hidden state，作为社交信息输入。
    """
    def __init__(self, input_dim=6, hidden_dim=128, pred_len=60, dropout=0.1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.pred_len = pred_len

        # 嵌入原始输入特征
        self.embed = nn.Linear(input_dim, hidden_dim)

        # 社交交互 + LSTM 单元
        self.lstm_cell = nn.LSTMCell(hidden_dim * 2, hidden_dim)

        # 最终预测头
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, pred_len * 2)
        )

    def forward(self, data):
        """
        data.x: shape (B*A, T, D)
        data.num_graphs = B
        """
        x = data.x  # (B*A, T, D)
        B = data.num_graphs
        A = x.size(0) // B
        T = x.size(1)

        # 重新组织形状
        x = x.view(B, A, T, self.input_dim)  # (B, A, T, D)

        # 嵌入输入
        x_embed = self.embed(x)  # (B, A, T, hidden_dim)

        # 初始化 hidden 和 cell
        h = torch.zeros(B, A, self.hidden_dim, device=x.device)
        c = torch.zeros(B, A, self.hidden_dim, device=x.device)

        # 时序迭代
        for t in range(T):
            x_t = x_embed[:, :, t, :]  # (B, A, hidden_dim)

            # 社交池化：对所有 agent 的 h 做平均
            social_context = h.mean(dim=1, keepdim=True).expand(-1, A, -1)  # (B, A, hidden_dim)

            # 拼接个体状态 + 社交特征
            lstm_input = torch.cat([x_t, social_context], dim=-1)  # (B, A, 2*hidden_dim)

            # LSTMCell 只能处理 2D，所以展平后逐 agent 处理
            h = h.view(B * A, self.hidden_dim)
            c = c.view(B * A, self.hidden_dim)
            lstm_input = lstm_input.view(B * A, -1)

            h, c = self.lstm_cell(lstm_input, (h, c))  # 单步更新
            h = h.view(B, A, self.hidden_dim)
            c = c.view(B, A, self.hidden_dim)

        # 最终只取 ego agent 的 hidden state
        ego_h = h[:, 0, :]  # (B, hidden_dim)

        out = self.fc(ego_h)              # (B, pred_len*2)
        out = out.view(B, self.pred_len, 2)  # (B, pred_len, 2)

        return out


# ====== 测试用例 ======
if __name__ == "__main__":
    class DummyData:
        pass

    B, A, T, D = 4, 50, 50, 6
    pred_len = 60
    torch.manual_seed(123)
    dummy = DummyData()
    dummy.x = torch.randn(B * A, T, D)
    dummy.num_graphs = B

    model = SocialLSTM(input_dim=D, hidden_dim=128, pred_len=pred_len, dropout=0.1)
    output = model(dummy)
    print(output.shape)  # 应输出: torch.Size([4, 60, 2])

torch.Size([4, 60, 2])


In [46]:
model = SocialLSTM(
    input_dim=6,
    hidden_dim=256,
    pred_len=60,
    dropout=0.1,
).to(device)


optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.75) # You can try different schedulers
early_stopping_patience = 10
best_val_loss = float('inf')
no_improvement = 0
criterion = nn.MSELoss()

In [47]:
for epoch in tqdm.tqdm(range(100), desc="Epoch", unit="epoch"):
    # ---- Training ----
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(batch.num_graphs, 60, 2)
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        train_loss += loss.item()

    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = batch.to(device)
            pred = model(batch)
            y = batch.y.view(batch.num_graphs, 60, 2)
            val_loss += criterion(pred, y).item()

            # show MAE and MSE with unnormalized data
            pred = pred * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            y = y * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            val_mae += nn.L1Loss()(pred, y).item()
            val_mse += nn.MSELoss()(pred, y).item()

    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    scheduler.step()
    # scheduler.step(val_loss)

    tqdm.tqdm.write(f"Epoch {epoch:03d} | Learning rate {optimizer.param_groups[0]['lr']:.6f} | train normalized MSE {train_loss:8.4f} | val normalized MSE {val_loss:8.4f}, | val MAE {val_mae:8.4f} | val MSE {val_mse:8.4f}")
    if val_loss < best_val_loss - 1e-3:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        no_improvement += 1
        if no_improvement >= early_stopping_patience:
            print("Early stop!")
            break

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]<ipython-input-16-3024836103>:33: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future - origin
<ipython-input-16-3024836103>:47: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future @ R
<ipython-input-16-3024836103>:24: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future @ R
Epoch:   1%|          | 1/100 [00:26<43:56, 26.63s/epoch]

Epoch 000 | Learning rate 0.001000 | train normalized MSE  64.3009 | val normalized MSE  23.5604, | val MAE   2.8736 | val MSE  23.5604


Epoch:   2%|▏         | 2/100 [00:53<43:43, 26.77s/epoch]

Epoch 001 | Learning rate 0.001000 | train normalized MSE  24.4762 | val normalized MSE  22.3684, | val MAE   2.6976 | val MSE  22.3684


Epoch:   3%|▎         | 3/100 [01:20<43:18, 26.79s/epoch]

...

Epoch 057 | Learning rate 0.000563 | train normalized MSE  12.5996 | val normalized MSE  11.7170, | val MAE   1.7394 | val MSE  11.7170


Epoch:  59%|█████▉    | 59/100 [26:19<18:17, 26.78s/epoch]

Epoch 058 | Learning rate 0.000563 | train normalized MSE  12.7313 | val normalized MSE  13.0366, | val MAE   1.8004 | val MSE  13.0366


Epoch:  59%|█████▉    | 59/100 [26:45<18:35, 27.21s/epoch]

Epoch 059 | Learning rate 0.000422 | train normalized MSE  12.8282 | val normalized MSE  11.7551, | val MAE   1.7402 | val MSE  11.7551
Early stop!


In [49]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

best_model = torch.load("best_model.pt")
model = SocialLSTM(
    input_dim=6,
    hidden_dim=256,
    pred_len=60,
    dropout=0.1,
).to(device)

model.load_state_dict(best_model)
model.eval()

pred_list = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        pred_norm = model(batch)

        # Reshape the prediction to (N, 60, 2)
        pred = pred_norm * batch.scale.view(-1,1,1) + batch.origin.unsqueeze(1)
        pred_list.append(pred.cpu().numpy())
pred_list = np.concatenate(pred_list, axis=0)  # (N,60,2)
pred_output = pred_list.reshape(-1, 2)  # (N*60, 2)
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])
output_df.index.name = 'index'
output_df.to_csv('submission.csv', index=True)

In [9]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR, LambdaLR

# -------------------
# 1) Positional Encoding 同上，不变
# -------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2, dtype=torch.float32) * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]


# -------------------
# 2) Social Attention with Top‐K neighbors + 双层
# -------------------
class SocialTransformerAttention(nn.Module):
    def __init__(self, hidden_dim: int, num_heads: int = 8, dropout: float = 0.1, topk: int = 10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.topk = topk

        self.attn = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads,
            batch_first=True,
            dropout=dropout
        )
        self.pos_proj = nn.Linear(3, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 4, hidden_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, ego_feat, other_feats, ego_pos, other_pos, mask=None):
        """
        ego_feat  : (B, H)
        other_feats: (B, N, H)
        ego_pos   : (B, 2)
        other_pos : (B, N, 2)
        mask      : (B, N) - True 表示屏蔽
        """
        B, N, H = other_feats.shape

        # 1) 先用 Top‐K 的方式动态挑最近的 neighbors
        #    如果 mask 已经标了很远/全零的也在 True，那只管还剩多少？我们尽量从剩余的里再取 topK
        with torch.no_grad():
            dist_all = torch.norm(other_pos - ego_pos.unsqueeze(1), dim=-1)  # (B, N)
            if mask is not None:
                dist_all = dist_all.masked_fill(mask, float('inf'))  # 把被 mask 的设为 +∞
            # 取最近 topk 个
            topk_vals, topk_idx = torch.topk(dist_all, k=min(self.topk, N), largest=False)  # (B, topk)

            # 构造一个 new_mask，使得 **不** 在 topk_idx 内的都变 True
            new_mask = torch.ones((B, N), device=other_feats.device, dtype=torch.bool)
            arange_b = torch.arange(B, device=other_feats.device).unsqueeze(1)  # (B,1)
            new_mask[arange_b, topk_idx] = False
            # 合并之前的 mask
            if mask is not None:
                mask = mask | new_mask
            else:
                mask = new_mask

        # 2) Query / Key / Value
        q = ego_feat.unsqueeze(1)  # (B,1,H)
        k = other_feats            # (B,N,H)
        v = other_feats            # (B,N,H)

        # 3) 相对位置编码
        rel_pos = other_pos - ego_pos.unsqueeze(1)       # (B,N,2)
        dist = torch.norm(rel_pos, dim=-1, keepdim=True) # (B,N,1)
        pos_embed = torch.cat([rel_pos, dist], dim=-1)   # (B,N,3)
        pos_bias = self.pos_proj(pos_embed)              # (B,N,H)
        k = k + pos_bias
        v = v + pos_bias

        # 4) 防止全 mask 导致 nan
        all_masked = mask.all(dim=1)
        if all_masked.any():
            mask[all_masked, 0] = False

        # 5) MultiHeadAttention
        attn_out, _ = self.attn(q, k, v, key_padding_mask=mask)  # (B,1,H)
        out = attn_out.squeeze(1)                                 # (B,H)
        out = self.ln1(ego_feat + self.dropout(out))

        # 6) FFN + 残差 + LayerNorm
        ff = self.ffn(out)
        out = self.ln2(out + self.dropout(ff))

        return out  # (B,H)


# -------------------
# 3) 主干网络：更深的 TransformerEncoder + 2 层 Social Attention + 融合 FFN
# -------------------
class SocialTransformerPredictor(nn.Module):
    def __init__(self,
                 input_dim: int = 5,      # 现在你只用 (x,y,vx,vy,ax) 共5维
                 hidden_dim: int = 512,   # 加大到 512
                 pred_len: int = 60,
                 num_agents: int = 50,
                 seq_len: int = 50,
                 num_heads: int = 4,
                 enc_layers: int = 4,     # 深度改成 4
                 dropout: float = 0.1,
                 neighbor_threshold: float = 10.0,
                 scale: float = 1.0,
                 topk: int = 10):         # 只选最近 10 个邻居
        super().__init__()
        self.hidden_dim = hidden_dim
        self.pred_len = pred_len
        self.num_agents = num_agents
        self.seq_len = seq_len
        self.neighbor_threshold = neighbor_threshold
        self.scale = scale

        # 3.1) 时序编码：Linear→PosEnc→TransformerEncoder
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.pos_enc = PositionalEncoding(d_model=hidden_dim, max_len=seq_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=enc_layers
        )
        self.temporal_ln = nn.LayerNorm(hidden_dim)

        # 3.2) Social Attention：两层串联
        self.social_attn1 = SocialTransformerAttention(
            hidden_dim=hidden_dim,
            num_heads=num_heads,
            dropout=dropout,
            topk=topk
        )
        self.social_attn2 = SocialTransformerAttention(
            hidden_dim=hidden_dim,
            num_heads=num_heads,
            dropout=dropout,
            topk=topk
        )

        # 3.3) 融合后再过一个小 FFN
        self.fusion_ln = nn.LayerNorm(hidden_dim * 2)
        self.fusion_dropout = nn.Dropout(dropout)
        self.fusion_ffn = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, hidden_dim * 2)
        )

        # 3.4) 最终 MLP Head
        self.head = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 4, pred_len * 2)
        )

    def forward(self, data):
        """
        data.x          : (B, A, T, 5)
        data.num_graphs:  B
        data.current_pos: (B, A, 2)
        """
        x = data.x
        B = data.num_graphs

        # 1) reshape → (B, A, T, 5)
        x = x.view(B, self.num_agents, self.seq_len, -1)

        # 2) 提取当前帧位置 current_pos
        if hasattr(data, 'current_pos'):
            current_pos = data.current_pos       # (B,50,2)
        else:
            current_pos = x[:, :, -1, :2].contiguous()  # (B,50,2)

        # 3) 时序编码：Linear→PosEnc→TransformerEncoder
        x_flat    = x.view(B * self.num_agents, self.seq_len, -1)  # (B*50,50,5)
        x_proj    = self.input_proj(x_flat)                        # (B*50,50,512)
        x_enc     = self.pos_enc(x_proj)                            # (B*50,50,512)
        x_encoded = self.transformer_encoder(x_enc)                 # (B*50,50,512)

        # 4) 取最后一帧特征（也可以替换为 max pooling，或 concat 多种时序池化）
        last_feat = x_encoded[:, -1, :]             # (B*50,512)
        feats     = last_feat.view(B, self.num_agents, self.hidden_dim)  # (B,50,512)
        feats     = self.temporal_ln(feats)         # LayerNorm

        # 5) 分离 ego 与 others
        ego_feat    = feats[:, 0, :]    # (B,512)
        other_feats = feats[:, 1:, :]   # (B,49,512)

        ego_pos     = current_pos[:, 0, :]   # (B,2)
        other_pos   = current_pos[:, 1:, :]  # (B,49,2)

        # 6) 构造距离 dist & TopK 掩码
        dist_all = torch.norm(other_pos - ego_pos.unsqueeze(1), dim=-1)  # (B,49)
        threshold = self.neighbor_threshold / self.scale
        hard_mask = dist_all > threshold                 # (B,49)
        base_mask = hard_mask.to(feats.device)           # 只用距离mask

        # 7) 第一层 Social Attention
        updated_1 = self.social_attn1(
            ego_feat, other_feats, ego_pos, other_pos, mask=base_mask
        )  # (B,512)

        # 8) 第二层 Social Attention （用刚刚 updated_1 做新 ego_feat）
        updated_2 = self.social_attn2(
            updated_1, other_feats, ego_pos, other_pos, mask=base_mask
        )  # (B,512)

        # 9) 融合：把原始 ego_feat 与 updated_2 拼接
        fusion = torch.cat([ego_feat, updated_2], dim=-1)  # (B,1024)
        fusion = self.fusion_ln(fusion)
        fusion = self.fusion_dropout(fusion)
        ff_out = self.fusion_ffn(fusion)
        fusion = F.relu(fusion + ff_out)  # (B,1024)

        # 10) Head → (B, pred_len*2) → reshape → (B, pred_len, 2)
        out = self.head(fusion)
        out = out.view(B, self.pred_len, 2)
        return out


In [12]:
model = SocialTransformerPredictor(
    input_dim=6,
    hidden_dim=256,
    pred_len=60,
    num_agents=50,
    seq_len=50,
    num_heads=2,
    enc_layers=2,
    dropout=0.1,
    neighbor_threshold=10.0,
    scale=1.0,
    topk=10
).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.75) # You can try different schedulers
early_stopping_patience = 10
best_val_loss = float('inf')
no_improvement = 0
criterion = nn.MSELoss()

In [13]:
for epoch in tqdm.tqdm(range(100), desc="Epoch", unit="epoch"):
    # ---- Training ----
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(batch.num_graphs, 60, 2)
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        train_loss += loss.item()

    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = batch.to(device)
            pred = model(batch)
            y = batch.y.view(batch.num_graphs, 60, 2)
            val_loss += criterion(pred, y).item()

            # show MAE and MSE with unnormalized data
            pred = pred * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            y = y * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            val_mae += nn.L1Loss()(pred, y).item()
            val_mse += nn.MSELoss()(pred, y).item()

    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    scheduler.step()
    # scheduler.step(val_loss)

    tqdm.tqdm.write(f"Epoch {epoch:03d} | Learning rate {optimizer.param_groups[0]['lr']:.6f} | train normalized MSE {train_loss:8.4f} | val normalized MSE {val_loss:8.4f}, | val MAE {val_mae:8.4f} | val MSE {val_mse:8.4f}")
    if val_loss < best_val_loss - 1e-3:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        no_improvement += 1
        if no_improvement >= early_stopping_patience:
            print("Early stop!")
            break

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]<ipython-input-4-2409732783>:39: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future - origin
<ipython-input-4-2409732783>:30: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future @ R
Epoch:   1%|          | 1/100 [00:28<47:31, 28.80s/epoch]

Epoch 000 | Learning rate 0.000100 | train normalized MSE  92.9601 | val normalized MSE  18.3528, | val MAE   2.4280 | val MSE  18.3528


Epoch:   2%|▏         | 2/100 [00:57<46:51, 28.69s/epoch]

Epoch 001 | Learning rate 0.000100 | train normalized MSE  22.0859 | val normalized MSE  14.2802, | val MAE   2.1195 | val MSE  14.2802


Epoch:   3%|▎         | 3/100 [01:26<46:21, 28.67s/epoch]

...

Epoch 031 | Learning rate 0.000075 | train normalized MSE  11.8292 | val normalized MSE  12.1899, | val MAE   1.8424 | val MSE  12.1899


Epoch:  33%|███▎      | 33/100 [15:44<31:55, 28.59s/epoch]

Epoch 032 | Learning rate 0.000075 | train normalized MSE  11.7976 | val normalized MSE  10.7061, | val MAE   1.6482 | val MSE  10.7061


Epoch:  33%|███▎      | 33/100 [16:12<32:54, 29.48s/epoch]

Epoch 033 | Learning rate 0.000075 | train normalized MSE  11.6355 | val normalized MSE  10.9036, | val MAE   1.6445 | val MSE  10.9036
Early stop!


In [14]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

best_model = torch.load("best_model.pt")
model = SocialTransformerPredictor(
    input_dim=6,
    hidden_dim=256,
    pred_len=60,
    num_agents=50,
    seq_len=50,
    num_heads=2,
    enc_layers=2,
    dropout=0.1,
    neighbor_threshold=10.0,
    scale=1.0,
    topk=10
).to(device)

model.load_state_dict(best_model)
model.eval()

pred_list = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        pred_norm = model(batch)

        # Reshape the prediction to (N, 60, 2)
        pred = pred_norm * batch.scale.view(-1,1,1) + batch.origin.unsqueeze(1)
        pred_list.append(pred.cpu().numpy())
pred_list = np.concatenate(pred_list, axis=0)  # (N,60,2)
pred_output = pred_list.reshape(-1, 2)  # (N*60, 2)
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])
output_df.index.name = 'index'
output_df.to_csv('submission.csv', index=True)